In [7]:
import numpy as np
import scipy.io
import cv2
import matplotlib.pyplot as plt
import os


In [8]:
# Specify the path to the data folder containing the images
data_folder = r"C:\Users\Nagham\HRNet-Human-Pose-Estimation\data\hr-lspet1"  # Replace with the actual path to your image folder

In [9]:
# Load the annotation file
#annotations = scipy.io.loadmat(r"C:\Users\Nagham\HRNet-Human-Pose-Estimation\data\hr-lspet\joints.mat")
# Load the MATLAB file
annotations = scipy.io.loadmat(os.path.join(data_folder, 'joints.mat'))
joints = annotations['joints']  # Shape will be (3, 14, N), where N is the number of images


In [17]:
# Check the type and shape of the joints array
print("Type of joints:", type(joints))
print("Shape of joints:", joints.shape)

Type of joints: <class 'numpy.ndarray'>
Shape of joints: (14, 3, 9428)


In [15]:
# Example: Extract keypoints for the first image
keypoints_first_image = joints[:, :, 3817].T  # Extract x and y coordinates and transpose to get shape (14, 2)
print("Keypoints for the first image:\n", keypoints_first_image)

Keypoints for the first image:
 [[156  82 169 220 145  91 420 359 304 330 360 422 325 375]
 [248 279 234 280 292 300  68 106 146 227 284 279 166 170]
 [  0   1   1   1   1   1   1   1   1   1   1   1   1   1]]


In [16]:
# Example: Extract keypoints for the first image
keypoints_first_image = joints[:, :, 0]  # Extract x and y coordinates and the visibility tag
print("Keypoints for the first image:\n", keypoints_first_image)

Keypoints for the first image:
 [[156 248   0]
 [ 82 279   1]
 [169 234   1]
 [220 280   1]
 [145 292   1]
 [ 91 300   1]
 [420  68   1]
 [359 106   1]
 [304 146   1]
 [330 227   1]
 [360 284   1]
 [422 279   1]
 [325 166   1]
 [375 170   1]]


In [ ]:
def get_bounding_box(keypoints, padding=0):
    """
    Calculate the bounding box for a given set of keypoints.
    
    Parameters:
    keypoints (array-like): An array of keypoints with shape (N, 2), where N is the number of keypoints.
    padding (int): Optional padding to add around the bounding box.
    
    Returns:
    tuple: The coordinates of the bounding box (x_min, y_min, x_max, y_max).
    """
    keypoints = np.array(keypoints)
    x_min = np.min(keypoints[:, 0])
    x_max = np.max(keypoints[:, 0])
    y_min = np.min(keypoints[:, 1])
    y_max = np.max(keypoints[:, 1])
    
    # Add padding
    x_min -= padding
    x_max += padding
    y_min -= padding
    y_max += padding
    
    return x_min, y_min, x_max, y_max

In [ ]:
# Example keypoints for a single person
keypoints = joints[:, :, 0]  # Extract x and y coordinates and the visibility tag

# Calculate the bounding box with a padding of 5 pixels
bounding_box = get_bounding_box(keypoints, padding=5)

print("Bounding Box Coordinates:", bounding_box)


In [ ]:
for i in range(joints.shape[2]):
    keypoints = joints[:, :2, i]  # Extract keypoints for the i-th image and transpose to get shape (14, 2)
    bounding_box = get_bounding_box(keypoints, padding=5)
    print(f"Image {i+1} Bounding Box: {bounding_box}")

In [ ]:
def draw_bounding_box(image, bbox, color=(0, 255, 0), thickness=2):
    """
    Draw a bounding box on the image.
    
    Parameters:
    - image: The image on which to draw.
    - bbox: A tuple (x_min, y_min, x_max, y_max) representing the bounding box.
    - color: The color of the bounding box.
    - thickness: The thickness of the bounding box lines.
    """
    x_min, y_min, x_max, y_max = bbox
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, thickness)
    return image

In [ ]:
# Load the image file names
#image_files = [f'im{i+1:05d}.jpg' for i in range(joints.shape[2])]  # Assuming images are named as 'im0001.jpg', 'im0002.jpg', etc.
# Load the image file names
image_files = [os.path.join(data_folder, f'im{i+1:05d}.png') for i in range(10)]  # Assuming images are named as 'im0001.jpg', 'im0002.jpg', etc.

In [ ]:
# Load and display the images with bounding boxes
for i, image_file in enumerate(image_files):
    # Load the image
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for displaying with Matplotlib
    
    # Get the keypoints for the current image
    keypoints = joints[:, :2, i]  # Extract x and y coordinates and transpose to get shape (14, 2)
    
    # Calculate the bounding box
    bbox = get_bounding_box(keypoints, padding=15)
    
    # Draw the bounding box on the image
    image_with_bbox = draw_bounding_box(image, bbox)
    
    # Display the image
    plt.figure(figsize=(6, 6))
    plt.imshow(image_with_bbox)
    plt.title(f'Image {i+1} with Bounding Box')
    plt.axis('off')
    plt.show()

In [ ]:
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt

# Load pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define image transformations
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [ ]:
# Load and display the images with bounding boxes
for i, image_file in enumerate(image_files):
    # Load the image
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for displaying with Matplotlib
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        predictions = model(image_tensor)

    # Extract relevant information from predictions
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    
  # COCO dataset class ID for "person" is 1
    PERSON_CLASS_ID = 1
  # Set a confidence threshold
    confidence_threshold = 0.5

    # Filter boxes for humans with scores above the threshold
    human_boxes = [box for box, label, score in zip(boxes, labels, scores) if label == PERSON_CLASS_ID and score > confidence_threshold]

    
    # Plot the image and draw the bounding boxes
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    # Draw bounding boxes
    for box in human_boxes:
        xmin, ymin, xmax, ymax = box
        width, height = xmax - xmin, ymax - ymin
        rect = plt.Rectangle((xmin, ymin), width, height, fill=False, color='red', linewidth=2)
        ax.add_patch(rect)

    plt.show()